In [1]:

from transformers import pipeline
sega = pipeline("text2text-generation",model='beyond/sega-large', device=1)

In [ ]:
'<mask> Conference on Empirical Methods <mask> submission of research papers <mask> Deep Learning <mask>'
'Bad news: <mask> the European Union <mask> month by EU <mask> Farm Commissioner Franz <mask>'

ss = ['nice weekend movie fun',
      'Shanghai food water help citizens virus',
      'England ship China ocean Paris cake people',
      'joint great food great drinks greater staff',
      'Wuhan hot-dry noodel delicious breakfirst China street Hubuxiang tour nice place',]

In [7]:
from transformers import pipeline
# sega = pipeline("text2text-generation",model='saved_models/bart-base-c4-realnewslike-4templates-passage-max15sents_2-sketch4/checkpoint-129375', framework='pt')
s = '<mask> machine learning <mask> my research interest <mask> data science <mask>'
sega(s, num_beams=3, do_sample=True, max_length=100)

[{'generated_text': 'I am interested in machine learning, and my research interest is in data science. I am a graduate student at the University of California, Davis.'}]

In [1]:
# sega-chinese
from transformers import BertTokenizer, BartForConditionalGeneration, Text2TextGenerationPipeline
model_path = 'saved_models/bart-base-chinese-chinese_clean_passages_80m_with_sketch-10000000/checkpoint-93750'
tokenizer = BertTokenizer.from_pretrained(model_path)
sega_model = BartForConditionalGeneration.from_pretrained(model_path)
sega_generator = Text2TextGenerationPipeline(sega_model, tokenizer, device=7)

bart_model = BartForConditionalGeneration.from_pretrained('fnlp/bart-base-chinese')
bart_generator = Text2TextGenerationPipeline(bart_model, tokenizer, device=6)

2022-11-07 09:47:24.890192: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [6]:
bart_generator("今天[MASK]篮球[MASK]上海财经大学[MASK]", max_length=50, do_sample=False)

[{'generated_text': '今 天 的 篮 球 是 上 海 财 经 大 学 篮 球'}]

In [7]:
sega_generator("今天[MASK]篮球[MASK]上海财经大学[MASK]", max_length=50, do_sample=False)

[{'generated_text': '今 天 的 篮 球 比 赛 是 由 上 海 财 经 大 学 举 办 的 。'}]

In [ ]:
sketchs = [
    # "今天[MASK]篮球[MASK]学校[MASK]",
    # "自然语言处理[MASK]谷歌[MASK]通用人工智能[MASK]",
    # "我昨天做了一个梦[MASK]又遇见了她[MASK]曾经那段时光让人怀恋[MASK]",
    "[MASK]疫情[MASK]公园[MASK]散步[MASK]",
    # "[MASK]酸菜鱼火锅[MASK]很美味，味道绝了[MASK]周末真开心[MASK]"
    ""
]
for sketch in sketchs:
    print('input sketch:\n>>> ', sketch)
    print('BART-chinese output:\n>>> ',bart_generator(sketch, max_length=100, do_sample=False)[0]['generated_text'].replace(' ',''))
    print('SEGA-chinese output:\n>>> ',sega_generator(sketch, max_length=100, do_sample=True, num_beams=3)[0]['generated_text'].replace(' ',''),'\n')


In [ ]:
from datasets import load_dataset
dataset_name = 'conll2003'
# dataset_name = 'wikiann'
raw_datasets = load_dataset(dataset_name)
raw_datasets

In [ ]:
from transformers import pipeline
sega = pipeline('text2text-generation', model='beyond/sega-base-ps')

In [20]:
from collections import defaultdict
import numpy as np

# 合并多条样本
def concat_multiple_sequences(dataset, size=3, overlap=True):
    # 传入正经的huggingface dataset格式
    # 如果是子集的话，建议使用select方法来筛选
    new_dataset = defaultdict(list)
    l = len(dataset)
    if overlap: # 连续窗口滑动
        for i in range(l-size):
            concat_tokens = np.concatenate(dataset[i:i+size]['tokens'])
            concat_tags = np.concatenate(dataset[i:i+size]['ner_tags'])
            new_dataset['tokens'].append(concat_tokens)
            new_dataset['ner_tags'].append(concat_tags)
    else:  # 互相不重叠
        for i in range(l//size):
            concat_tokens = np.concatenate(dataset[i*size:(i+1)*size]['tokens'])
            concat_tags = np.concatenate(dataset[i*size:(i+1)*size]['ner_tags'])
            new_dataset['tokens'].append(concat_tokens)
            new_dataset['ner_tags'].append(concat_tags)
    return new_dataset


tag_names = raw_datasets['train'].features['ner_tags'].feature.names

def get_mention_name(tag):
    # tag: the number/index of the tag name
    # tag_names: the list of tag names
    # mention: ORG, LOC, etc.
    return tag_names[tag].split('-')[-1]

# 单独把实体抽出来
def extract_mentions(tokens, tags):
    """
    return: 
    mentions: []
    mention_dict: {'MISC': [], 'PER': [], 'LOC': [], 'ORG': []}
    """
    mentions = []
    mention_dict = {t:[] for t in list(set([t.split('-')[-1] for t in tag_names])) if t != 'O'}
    for i in range(len(tokens)):
        mention = get_mention_name(tags[i])
        if mention == 'O':
            continue
        if tags[i] % 2 == 1:
            # the start
            mention_dict[mention].append([tokens[i]])
            mentions.append([tokens[i]])
        else:
            # the remaining part
            mention_dict[mention][-1].append(tokens[i])
            mentions[-1].append(tokens[i])
    for k in mention_dict:
        if mention_dict[k]: # not empty
            mention_dict[k] = [' '.join(items) for items in mention_dict[k]]
    mentions = [' '.join(items) for items in mentions]
    return mentions,mention_dict
    

def get_spans(tokens,window=3):
    spans = []
    for i in range(len(tokens) // window):
        spans.append(' '.join(tokens[i*window:(i+1)*window]))
    return spans

def extract_mention_spans(tokens, tags):
    """
    把一个句子中"""
    text = ' '.join(tokens)
    kws = keynotes_yake(text, 2, 5)
    spans = get_spans(tokens, window=3)
    mentions, _ = extract_mentions(tokens, tags)
    wanted_spans = []
    for span in spans[1:-1]:
        for w in mentions+[]:
            if w in span:
                wanted_spans.append(span)
                break
    m = '<mask> ' + ' <mask> '.join(wanted_spans) + ' <mask>'
    m = f"{spans[0]} {m} {spans[-1]}" # 相当于限制了边界
    return wanted_spans, m

from torch.utils.data import Dataset
class MDataset(Dataset):
    def __init__(self, m_list):
        self.masked_contents = m_list
    def __len__(self):
        return len(self.masked_contents)
    def __getitem__(self, i):
        return self.masked_contents[i]

In [8]:
orig_data = raw_datasets['train'].select(range(100))
orig_data

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 100
})

In [27]:
# concat_dataset = concat_multiple_sequences(orig_data)
concat_dataset['tokens'][0]

array(['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British',
       'lamb', '.', 'Peter', 'Blackburn', 'BRUSSELS', '1996-08-22'],
      dtype='<U10')